In [ ]:
import pandas as pd
import os
import sys

sys.path.append('..')
from utils.news_download_utils import download_newspaper4k, download_playwright, download_cloudscraper

In [ ]:
topics = ['immigration']

In [ ]:
# first step: download through newspaper4k

for topic in topics:
    print(f'Topic: {topic}')
    filename = f'news/retrieved/step1_{topic}.csv'
    if os.path.exists(filename):
        print(f'Skipping {filename}, already exists.')
        continue    
    news = pd.read_csv(f'news/merged/{topic}.csv')
    news = news.dropna(subset=['url'])

    downloaded_news = {}
    news['text'] = news['url'].apply(lambda x: download_newspaper4k(x, downloaded_news=downloaded_news))
    news.to_csv(f'news/retrieved/step1_{topic}.csv', index=False)


In [ ]:
# second step: download through playwright
for topic in topics:
    print(f'Topic: {topic}')
    filename = f'news/retrieved/step2_{topic}.csv'
    if os.path.exists(filename):
        print(f'Skipping {filename}, already exists.')
        continue    
    news = pd.read_csv(f'news/retrieved/step1_{topic}.csv')
    # remove rows with 404
    news = news[news['text'] != 404].reset_index(drop=True)

    downloaded_news = {}
    for index, row in news.iterrows():
        if str(row['text']).isnumeric() or pd.isna(row['text']):
            news.at[index, 'text'] = await download_playwright(row['url'], downloaded_news=downloaded_news)

    news.to_csv(f'news/retrieved/step2_{topic}.csv', index=False)

In [ ]:
# third step: download through cloudscraper
for topic in topics:
    print(f'Topic: {topic}')
    filename = f'news/retrieved/step3_{topic}.csv'
    if os.path.exists(filename):
        print(f'Skipping {filename}, already exists.')
        continue    
    news = pd.read_csv(f'news/retrieved/step2_{topic}.csv')

    downloaded_news = {}
    for index, row in news.iterrows():
        if str(row['text']).isnumeric() or pd.isna(row['text']) or row['text'] == "-1":
            news.at[index, 'text'] = await download_playwright(row['url'], downloaded_news=downloaded_news)

    news.to_csv(f'news/retrieved/step3_{topic}.csv', index=False)